# Review Instructions

Please review the MSv4 `field_and_source_xds` schema and the XRADIO interface (`ps['MSv4_name'].VISIBILITY.field_and_source_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `field_and_source_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1658760192

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data

## `field_and_source_xds` Schema
The FIELD, SOURCE, and EPHEMERIS tables in the MSv2 contain closely related information:
- **FIELD**: Field position for a source.
- **SOURCE**: Information about the source being observed (position, proper motion, etc.)
- **EPHEMERIS**: Ephemerides of the source.

These can be combined into a single dataset for MSv4 because it consists of a single field and consequently a single source[^1].

### Use Cases
The use cases considered during the design of the schema were:
- Single field observation (type=standard).
- Mosaic observation (type=standard).
- Ephemeris observation (type=ephemeris).
- Mosaic Ephemeris observation (type=ephemeris).

To satisfy these use cases, two types of `field_and_source_xds` were created: standard and ephemeris. The main difference is that the ephemeris type has a `FIELD_PHASE_OFFSET` data variable that is relative to the `SOURCE_POSITION/SOURCE_DIRECTION` data variable (contains the ephemerides and has a time axis), while the standard type has `FIELD_PHASE/DELAY/REFERENCE_CENTERS` and `SOURCE_POSITION` (has no time axis). The `SOURCE_POSITION/DIRECTION` is kept separate from the `FIELD_PHASE_OFFSET/CENTER` so that the intent `OBSERVE_TARGET#OFF_SOURCE` is supported and the ephemeris can be easily changed.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
  - 1.2.1) Are there data variables we need to add (for example, the JPL Horizons data has additional information such as the North pole position angle, etc., see [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data))?
  - 1.2.2) In some CASA ephemeris tables, there are table keywords such as `VS_CREATE`, `VS_DATE`, `VS_TYPE`, `VS_VERSION`, `MJD0`, `dMJD`, `earliest`, `latest`, `radii`, `meanradm`, `orb_per`, `rot_per`. Do we need any of these?
- 1.3) Is there a use case where the `FIELD_PHASE_CENTER` and `FIELD_DELAY_CENTER` would differ (i.e., do we need to store both)?
- 1.4) For interferometer observations, do we need to store the `FIELD_REFERENCE_CENTER` or can this be omitted (will still be present for Single dish)?
- 1.5) The ephemeris data is recorded in degrees, AU, and MJD. Should these be converted to radians, meters, and time (Unix)? Note that each data variable has measurement information attached to it. For example:
```Python
  frame: ICRS
  type: sky_coord
  units: ['deg', 'deg', 'AU']
```

- 1.6) For ephemeris observations, should we add the SOURCE_PROPER_MOTION if available?
- 1.7) Is the name `field_and_source_xds` sufficiently descriptive?
- 1.8) Should we also add the DOPPLER table information to the schema (if so, any idea where we can get an MSv2 with a DOPPLER table)?
- 1.9) Any naming suggestions or data layout?
- 1.10) Are the data variable descriptions in the schema spreadsheet correct?
- 1.11) What measures (https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014) should we attach to each of the following data variables

  - NORTH_POLE_POSITION_ANGLE (quantity?)
  - NORTH_POLE_ANGULAR_DISTANCE (quantity?)
  - SUB_OBSERVER_DIRECTION (earth_location?)
  - SUB_SOLAR_POSITION (earth_location?)
  - HELIOCENTRIC_RADIAL_VELOCITY (quantity?)
  - OBSERVER_PHASE_ANGLE (quantity?)
- 1.12) Can NORTH_POLE_POSITION_ANGLE and NORTH_POLE_ANGULAR_DISTANCE be combined into a single data variable?

### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `field_and_source_xds` schema, do you find the XARRAY interface intuitive and easy to use?



[^1]: This is inhereted from MSv2 that only allows a single source per field [https://casacore.github.io/casacore-notes/229.pdf, p35], though a source can appear in more than one field.


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 162-create-combined-field-source-and-ephemeris-dataset
cd xradio
pip install -e .
```

# ALMA Example

An ephemeris mosaic observation of the sun.

ALMA archive file downloaded: https://almascience.nrao.edu/dataPortal/2022.A.00001.S_uid___A002_X1003af4_X75a3.asdm.sdm.tar 

- Project: 2022.A.00001.S
- Member ous id (MOUS): uid://A001/X3571/X130
- Group ous id (GOUS): uid://A001/X3571/X131

CASA commands used to create the dataset:
```python
importasdm(asdm='uid___A002_X1003af4_X75a3.asdm.sdm',vis='uid___A002_X1003af4_X75a3.ms',asis='Ephemeris Antenna Station Receiver Source CalAtmosphere CalWVR',bdfflags=True,with_pointing_correction=True,convert_ephem2geo=True)

importasdm(asdm='22A-347.sb41163051.eb41573599.59642.62832791667.asdm',vis='22A-347.sb41163051.eb41573599.59642.62832791667.ms',asis='Ephemeris Antenna Station Receiver Source',with_pointing_correction=True,convert_ephem2geo=True)

mstransform(vis='ALMA_uid___A002_X1003af4_X75a3.split.ms',outputvis='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms',createmms=False,timeaverage=True,timebin='2s',timespan='scan',reindex=True)

import numpy as np
tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::FLAG_CMD',nomodify=False)
tb.removerows(np.arange(tb.nrows())) # tb.removerows(np.arange(76401))
tb.flush()
tb.done()

tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::POINTING',nomodify=False)
tb.removerows(np.arange(tb.nrows()))   #tb.removerows(np.arange(5617115))
tb.flush()
tb.done()
```


## Download Data

In [ ]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [ ]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

# Convert dataset

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_field",
    parallel=False,
    overwrite=True,
    ephemeris_interpolate=True
)

********************************************************************************
ALMA_uid___A002_X1003af4_X75a3.split.avg.ms
{'DATA_DESC_ID': array([0, 1, 2], dtype=int32), 'INTENT': array(['CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#OFF_SOURCE',
       'CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMBIENT',
       'CALIBRATE_ATMOSPHERE#HOT,CALIBRATE_WVR#HOT',
       'CALIBRATE_PHASE#ON_SOURCE,CALIBRATE_WVR#ON_SOURCE',
       'OBSERVE_TARGET#OFF_SOURCE', 'OBSERVE_TARGET#ON_SOURCE'],
      dtype=object), 'FIELD_ID': array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)}
pair (0, 'CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#OFF_SOURCE', 0)
$$$$
pair (0, 'CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#OFF_SOURCE', 2)
$$$$
pair (0, 'CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#OFF_SOURCE', 3)
$$$$
pair (0, 'CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#OFF_SOURCE', 4)
$$$$
pair (0, 'CALIBRATE_ATMOS

In [ ]:
import numpy as np
a = np.array(1)

a.ndim
a.dtype
a.item()

In [ ]:
import numpy as np

phase_deg = np.mod(np.linspace(0 ,720, 19), 360) - 180
print(phase_deg)
print( np.unwrap(phase_deg, period=360))

# Inspect Processing Set

In [ ]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

In [ ]:
ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_310'].attrs['partition_info']

In [ ]:
ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_310'].VISIBILITY.attrs["field_and_source_xds"].is_ephemeris

# Inspect field_and_source_xds: Standard Use case (non-ephemeris)

In [ ]:
standard_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_902"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
standard_field_and_source_xds

In [ ]:
standard_field_and_source_xds  # How to access field_and_source_xds.

In [ ]:
standard_field_and_source_xds.FIELD_PHASE_CENTER  # How to access field_and_source_xds datavariables. standard_field_and_source_xds['FIELD_PHASE_CENTER'] can also be used.

In [ ]:
standard_field_and_source_xds.FIELD_PHASE_CENTER.attrs  # How to access field_and_source_xds datavariables measures information stored in the attributes.

# Inspect field_and_source_xds: Ephemeris Use case (Mosaic) with line

In [ ]:
ephemeris_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_963"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
ephemeris_field_and_source_xds

In [ ]:
ephemeris_field_and_source_xds.FIELD_PHASE_CENTER

In [ ]:
ephemeris_field_and_source_xds.SOURCE_POSITION

In [ ]:
ephemeris_field_and_source_xds.SOURCE_POSITION.sel(sky_pos_label="dec")

In [ ]:
ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_963"
]

In [ ]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms"
out_file = "VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_scan",
    parallel=True,
    overwrite=True,
)

********************************************************************************
VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms
{'DATA_DESC_ID': array([0, 1, 2, 3], dtype=int32), 'INTENT': array(['SYSTEM_CONFIGURATION#UNSPECIFIED',
       'CALIBRATE_FLUX#UNSPECIFIED,CALIBRATE_BANDPASS#UNSPECIFIED,CALIBRATE_POL_ANGLE#UNSPECIFIED',
       'CALIBRATE_POL_LEAKAGE#UNSPECIFIED',
       'CALIBRATE_PHASE#UNSPECIFIED,CALIBRATE_AMPLI#UNSPECIFIED',
       'OBSERVE_TARGET#UNSPECIFIED'], dtype=object), 'FIELD_ID': array([   0,    3,    4, ..., 6496,  199, 6300], dtype=int32)}
(array([0, 1, 2, 3], dtype=int32),)
$$$$
(array(['SYSTEM_CONFIGURATION#UNSPECIFIED',
       'CALIBRATE_FLUX#UNSPECIFIED,CALIBRATE_BANDPASS#UNSPECIFIED,CALIBRATE_POL_ANGLE#UNSPECIFIED',
       'CALIBRATE_POL_LEAKAGE#UNSPECIFIED',
       'CALIBRATE_PHASE#UNSPECIFIED,CALIBRATE_AMPLI#UNSPECIFIED',
       'OBSERVE_TARGET#UNSPECIFIED'], dtype=object),)
$$$$
(array([   0,    3,    4, ..., 6496,  199, 6300], dtype=int32)

In [ ]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.zarr")
ps.summary()

In [ ]:
ps.keys()

In [ ]:
ps['VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6_4']

In [ ]:
import matplotlib.pyplot as plt 

pc1 = ps['VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6_4'].VISIBILITY.field_and_source_xds.FIELD_PHASE_CENTER
pc2 = ps['VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6_14'].VISIBILITY.field_and_source_xds.FIELD_PHASE_CENTER

plt.figure()
plt.scatter(pc1[:,0],pc1[:,1])
plt.scatter(pc2[:,0],pc2[:,1])
plt.show()

In [ ]:
'1959467+631424' '1959202+631424' '1958537+631424' '1958271+631424'
 '1958006+631424' '1957341+631424' '1957076+631424' '1956411+631424'
 '1956145+631424' '1955480+631424' '1955215+631424' '1954550+631424'

In [ ]:
import numpy as np
a = np.array([[3434, 3434, 3434, -42, 3434, 3434],
 [3435, 3435, 3435, -42, 3435, 3435],
 [3436, -42, 3436, 3436, 3436, 3436]])
np.max(a,axis=1)

In [ ]:
# # Simplified code from read_col_conversion src/xradio/vis/_vis_utils/_ms/_tables/read.py 
# #Get MSv2
# import graphviper #need to pip install graphviper
# graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

# from casacore import tables
# import numpy as np

# #Col to read
# col="FIELD_ID"
# #col = "TIME"

# #Open table
# tb_tool = tables.table("ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

# #Check if getcol works
# print('Using getcol', tb_tool.getcol('FIELD_ID'), tb_tool.getcol('FIELD_ID').dtype)
# col_dtype = tb_tool.getcol(col,0,1).dtype
# print('col_dtype using getcol',col_dtype)

# # col_dtype = np.array(tb_tool.col(col)[0]).dtype
# # print('col_dtype',col_dtype)
# for ts in tb_tool.iter(col, sort=False):
#     num_rows = ts.nrows()
    
#     # Create small temporary array to store the partial column
#     # tmp_arr = np.full((num_rows,), np.nan, dtype=col_dtype)
#     tmp_arr = np.full((num_rows,), 0, dtype=col_dtype)
    
#     # Note we don't use `getcol()` because it's less safe. See:
#     # https://github.com/casacore/python-casacore/issues/130#issuecomment-463202373
#     ts.getcolnp(col, tmp_arr)


# tb_tool.close()

In [4]:
from casacore import tables
import numpy as np
import xarray as xr
import  pandas as pd

pd.set_option("display.max_rows", 1000)

#Col to read
col="FIELD_ID"
#col = "TIME"

#Open table
main_tb = tables.table("VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms")
field_tb = tables.table("VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::FIELD")
source_tb = tables.table("VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::SOURCE")    
state_tb = tables.table("VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::STATE")  

#Check if getcol works
print('Using getcol', main_tb.getcol('FIELD_ID'), main_tb.getcol('FIELD_ID').dtype)

cols = ['DATA_DESC_ID','FIELD_ID','SCAN_NUMBER','STATE_ID']

df = pd.DataFrame()

for c in cols:
    df[c] = xr.DataArray(main_tb.getcol(c),dims=['row'])
    
#xds['intents']
df['SOURCE_ID'] = xr.DataArray(field_tb.getcol('SOURCE_ID')[df['FIELD_ID']],dims=['row'])
df['FIELD_NAME'] = xr.DataArray(np.array(field_tb.getcol('NAME'))[df['FIELD_ID']],dims=['row'])
df['SOURCE_NAME'] = xr.DataArray(np.array(source_tb.getcol('NAME'))[df['SOURCE_ID']],dims=['row'])  
df['INTENT'] = xr.DataArray(np.array(state_tb.getcol('OBS_MODE'))[df['STATE_ID']],dims=['row'])
df['SUB_SCAN_NUMBER'] = xr.DataArray(state_tb.getcol('SUB_SCAN')[df['STATE_ID']],dims=['row'])


# import matplotlib.pyplot as plt 
# plt.figure()
# plt.plot(xds['FIELD_ID'][0:200],marker='*')
# plt.plot(xds['SOURCE_ID'][0:200],marker='o')
# plt.show()

#main_tb .close()

df_sub = pd.DataFrame()

sel_cols = ['DATA_DESC_ID','FIELD_ID','SCAN_NUMBER','STATE_ID','SOURCE_ID','INTENT','SUB_SCAN_NUMBER','FIELD_NAME','SOURCE_NAME']
#sel_cols = ['FIELD_ID','SCAN_NUMBER','STATE_ID','SOURCE_ID','INTENT','SUB_SCAN_NUMBER','FIELD_NAME'] #,'SOURCE_NAME'


for c in sel_cols:
    df_sub[c] = df[c].values

# #df=df.drop(['FIELD_ID','SUB_SCAN_NUMBER','STATE_ID','SCAN_NUMBER','INTENT'],axis=1)
# df=df.drop(['FIELD_ID','SUB_SCAN_NUMBER','STATE_ID','SCAN_NUMBER','SOURCE_ID'],axis=1)
# df.drop_duplicates()
df_sub = df_sub.drop_duplicates()

#df_sub= df_sub[df_sub['INTENT'] == 'SYSTEM_CONFIGURATION#UNSPECIFIED']
df_sub= df_sub[df_sub['INTENT'] == 'OBSERVE_TARGET#UNSPECIFIED']

#df_sub['INTENT'].drop_duplicates()

#        'SYSTEM_CONFIGURATION#UNSPECIFIED', (3+1)
#        'CALIBRATE_FLUX#UNSPECIFIED,CALIBRATE_BANDPASS#UNSPECIFIED,CALIBRATE_POL_ANGLE#UNSPECIFIED', (1)
#        'CALIBRATE_POL_LEAKAGE#UNSPECIFIED', (1)
#        'CALIBRATE_PHASE#UNSPECIFIED,CALIBRATE_AMPLI#UNSPECIFIED', (1, with multiple scan numbers 15)
#        'OBSERVE_TARGET#UNSPECIFIED'], (12070)
print(df_sub.shape)
df_sub['SCAN_NUMBER'].drop_duplicates()

df_sub

Successful readonly open of default-locked table VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms: 22 columns, 294144 rows
Successful readonly open of default-locked table VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::FIELD: 13 columns, 12205 rows
Successful readonly open of default-locked table VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::SOURCE: 14 columns, 804 rows
Successful readonly open of default-locked table VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms::STATE: 7 columns, 199 rows
Using getcol [   0    0    0 ... 6300 6300 6300] int32


KeyError: 'SOURCE_NAME'

In [ ]:
import numpy as np

a = np.array([1, 2, 3, 4, 5, 7, 5, 5, 10])
np.where(a==5)